<a href="https://colab.research.google.com/github/cwagner2325/WiseInbox_ML/blob/main/Wise_Inbox_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio
!pip install scikit-learn

In [84]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import random
import os
from google.colab import userdata
from pyngrok import conf
from time import sleep
import sklearn
from sklearn.pipeline import Pipeline

In [101]:
app = FastAPI()

In [102]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [103]:
class model_input(BaseModel):

    Email : str
    Sender: str
    Subject : str

In [104]:
with open('/BodyModel1', 'rb') as f:
    model = pickle.load(f)

In [105]:
def predictEmail(input_text):
    prediction = model.predict([input_text])

    probabilities = model.predict_proba([input_text])

    predicted_class = prediction[0]
    conf_index = 0 if predicted_class == "Phishing Email" else 1
    confidence = probabilities[0][conf_index]

    print(confidence, "chance this is a", predicted_class)

    return predicted_class, confidence

In [106]:
@app.post('/email_prediction')
def email_prediction(input_parameters : model_input):
    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)

    emailBody = input_dictionary['Email']
    sender = input_dictionary['Sender']
    subject = input_dictionary['Subject']


    predictionResult, confidence = predictEmail(emailBody)

    if predictionResult == "Phishing Email":
        # If the model is 100% confident it's a phishing email, set the prediction to 50
        prediction = 75 - ((confidence - .5) * 60)
    else:
        # If the model is 100% confident it's a safe email, set the prediction to 100
        prediction = 75 + ((confidence - .5) * 60)

    return {"prediction": prediction}

In [107]:
os.environ["TOKEN"] = userdata.get("TOKEN")
ngrok.set_auth_token(os.environ["TOKEN"])

ngrok_tunnel = ngrok.connect(8000)

# print('Public URL:', ngrok_tunnel.public_url)
# print("Here is a test command to run in a terminal: ")
# print('curl --insecure -X POST -H "Content-Type: application/json" -d \'{\n' +
#       '\t"Email": "example@example.com",\n\t"Sender": "John Doe",\n\t"Subject": "Important News" \n}\' ' +
#       ngrok_tunnel.public_url + "/email_prediction")

print("Running at public URL:", ngrok_tunnel.public_url + "/email_prediction")

nest_asyncio.apply()
uvicorn.run(app, port=8000, log_level="warning")

Running at public URL: https://ed19-35-188-239-227.ngrok-free.app/email_prediction
Dear Valued Customer,

We understand the importance of keeping your account secure, which is why we're reaching out to you regarding a recent security review. During this process, we identified some unusual activity on your account, and we kindly ask you to verify your information to prevent any unauthorized access.

To ensure the safety of your account, please click on the following link and follow the instructions provided: https://secure-verificationportal.com

Please note that if you do not update your information within the next 48 hours, we may be forced to temporarily suspend your account until the verification process is complete.

We apologize for any inconvenience this may cause, but rest assured, we're committed to protecting your sensitive data and maintaining the highest level of security for our valued customers.

Thank you for your prompt attention to this matter.

Sincerely,
[Company Name